# Import the library

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

Import the dataset

In [ ]:
dataset = pd.read_csv("Churn_Modelling.csv")
X = dataset.iloc[:,3:-1].values
y = dataset.iloc[:,-1].values

# Encoding catagorical data

Label Encoding Gender



In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

print(X)

One Hot encoding geography

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers = [('encoder',OneHotEncoder(),[1])],remainder="passthrough")
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

Split the dataset into test and train set

In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.2 , random_state = 0)

Feature scaling

In [ ]:
from sklearn.preprocessing import StandardScaler


# scaling all the columns in the data as it is nessary in ANN
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Building the ANN

Initialize the ANN

In [ ]:
ann = tf.keras.models.Sequential() # this initializes the layers of the ANN

Adding the input layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Add a second layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Add a output layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

# Training the ANN

Compiling the ANN

In [ ]:
ann.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy']) # catagorical_cross entroy as loss for non binary outputs

Training the ANN

In [ ]:
ann.fit(X_train , y_train , batch_size = 32 , epochs = 100)

Predict outputs

In [ ]:
print(ann.predict(sc.transform([[1.0,0.0,0.0,600,1,40,3,60000,2,1,1,50000]])) > 0.5)

Predict on test case values

In [ ]:
y_pred = ann.predict(X_test)

In [ ]:
y_pred = (y_pred > 0.5)

Create a confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)
print(cm)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)